<a href="https://colab.research.google.com/github/robin-li/mysql-views-dependency-dump/blob/main/mysql_view_schema_dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 問題描述

Navicat是MySQL可視化工具中最棒的，但是，在處理視圖的導入導出方面，它是按照視圖名稱的字母順序來處理的，若視圖存在依賴，在導入過程中就會報錯。

### 解決方法

在MySql很容易查出所有視圖和其定義，因此可以寫一個視圖導出工具，存儲時對各視圖的保存位置進行調整，處理好它們之間的依賴關係，被依賴的放前面，這樣就解決了導入時的依賴問題。

### 獲取視圖信息

運行以下查詢語句，就能獲得該數據庫中所有視圖的信息。

```sql
select * from  information_schema.VIEWS where TABLE_SCHEMA = DatabaseName 
```

查詢結果字段說明：

- TABLE_NAME ： 數所庫中視圖名稱
- VIEW_DEFINITION ： 視圖的定義代碼，只有查詢語句部分
- DEFINER ： 視圖定義（建立）者名稱
- SECURITY ： 安全級別

總之，所有視圖的信息都在這個表中保存，我要完成任務，只需要TABLE\_NAME和VIEW\_DEFINITION就可以了。

### 算法描述

- 將查詢結果放到dict中，視圖名稱為key；視圖定義為value；
- 編寫處理依賴關係的函數process\_rely，輸入參數中的rely\_old為保存所有視圖名稱的數組；返回參數為按依賴關係調整順序後的視圖名稱數組。之所以這樣做，是一開始考慮到，依賴關係複雜時，可能一次迭代處理不好，需要遞歸調用或多次調用。
    
    > process_rely函數算法描述：
    
    - 第一層循環，從rely_old中取一個視圖名稱
        
        - 第二層循環，從dict中取出一個鍵值
            
            - 若鍵值被第一層元素的定義所依賴
                
                - 若鍵值還不在結果數組中
                    
                    - 若第一層元素不在結果數組中
                        
                        - 追加鍵值到結果數組中
                    - 第一層元素在結果數組中
                        
                        - 將鍵值插入到第一層元素前
                - 鍵值在結果數組中
                    
                    - 第一層元素在結果數組中
                        
                        - 查找各自在結果數組中的位置
                        - 若第一層元素在鍵值的後
                            
                            - 將鍵值移動到第一層元素前
        - 第二層循環結束時，若第一層元素還不在結果集中
            
            - 將第一層元素追加到結果集中
    - 返回結果集
        

算法一次就能將複雜的依賴關係處理好了。


----------------------

### 完整代碼

#### 獲取外網IP 

首先獲取外網IP，以將其加入數據服務RDS白名單。


In [ ]:
import requests

def get_external_ip():
    url = 'https://api.ipify.org'
    try:
        response = requests.get(url)
        return response.text
    except:
        return 'Error'

print(get_external_ip())


#### 安装PyMySQL

In [ ]:
!pip install PyMySQL


#### 主程式

> 記得修改必要參數


In [ ]:
import pymysql

## 請修改以下參數
mysql_host='demo.mysql.host'
mysql_user='user'
mysql_passwd='xxxxxx'
target_db='target_db'

# 建立連線
conn = pymysql.connect(host=mysql_host, port=3306, user=mysql_user,
                       passwd=mysql_passwd, db='information_schema', charset='utf8')

print("Connection: " , conn)

def process_rely(parmas={}, rely_old=[]):
    _rely = []
    _keys = list(parmas.keys())
    for k in rely_old:
        for bl in _keys:
            if str(parmas[k]).find(bl) > -1:
                if bl not in _rely:
                    if k not in _rely:
                        _rely.append(bl)
                    else:
                        i = _rely.index(k)
                        _rely.insert(i, bl)
                else:
                    if k in _rely:
                        i = _rely.index(k)
                        j = _rely.index(bl)
                        if i < j:
                            del _rely[j]
                            _rely.insert(i, bl)
        if k not in _rely:
            _rely.append(k)
    return _rely


cur = conn.cursor()
cur.execute('select TABLE_NAME, VIEW_DEFINITION from  information_schema.VIEWS where TABLE_SCHEMA = %s ', target_db)
print("Cursor: " , cur)
rs = cur.fetchall()
cur.close()
conn.close()

print("Views: ")
ps = {}
for al in rs:
    ps['`' + al[0] + '`'] = al[1]
    print('  [View] `' + al[0] + '`')

# print(ps)

rely = process_rely(ps, list(ps.keys()))
# rely = process_rely(ps, rely1)

file_object = open('view.sql', 'w')
for al in rely:
    file_object.write('DROP VIEW IF EXISTS ' + al + ';\n')
    file_object.write('CREATE OR REPLACE VIEW ' + al +
                      ' AS ' + ps[al] + ';\n\n')

file_object.close()

print('DONE !')
